In [1]:
import pandas as pd

from tqdm import tqdm
import warnings

from statsbombpy import sb

In [2]:
###########################
# This notebook uses the statsbombpy API to grab all events for all seasons of StatsBomb's open data. Note that this is a large volume of data - about 5 GB, and running the notebook might therefore be difficult due to network speed and data transfer constraints on the API. It is advisable to use the downloaded csv files available in the /data directory instead.
###########################

In [4]:
# Scrape all events from all records in StatsBomb's open data and store then in a dataframe called all_events
warnings.filterwarnings("ignore")

all_events = pd.DataFrame()
all_events["season_id"] = []
all_events["match_id"] = []

for season in list(sb.competitions().iterrows())[1:2]:
    for match in tqdm(sb.matches(competition_id=season[1].iloc[0], season_id=season[1].iloc[1]).iterrows()):
        try:
            all_events = pd.concat([all_events,
                                    sb.events(match_id=match[1].iloc[0], split=False, flatten_attrs=True)
                                   .assign(competition_id=season[1].iloc[3], season_id=season[1].iloc[7], match_id=match[1].iloc[0])], ignore_index=True)
        except Exception as e:
            print(e)

52it [01:01,  1.19s/it]


In [ ]:
all_events